In [1]:
import pandas as pd
import numpy as np

print("Reading data from dataset initiated...")

agecate_ds = open(r"C:\Users\Admin\24[7]_Project\processed_text\age_category_ds.txt","r",encoding="utf-8")

pre_sentences = []
classes = []

listofrows = agecate_ds.readlines()

for line in listofrows:
    temp_sent = line.split("++$++")
    pre_sentences.append(temp_sent[0])
    classes.append(int(temp_sent[1]))
    

print("Reading data from dataset completed...")
print("Initializing categories...")

categories = ["child","teenager","adult","middle-aged","senior"]


Reading data from dataset initiated...
Reading data from dataset completed...
Initializing categories...


In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

print("Loading stopwords for removal...")
stop_words = set(stopwords.words('english')) 


def remove_punct(strin):
    punct_str = '!"#$%&\'()*+,-./:;?@[\]^_`{|}~'
    restr = ""
    for letter in strin:
        if(letter not in punct_str):
            restr = restr+letter
    return restr

pre_docs = []

for lines in pre_sentences:
    temp_sent = remove_punct(lines)
    sent_arr = temp_sent.split()
    clean_sent = ""
    for words in sent_arr:
        if words not in stop_words and len(words)>1:
            clean_sent+=words+" "
    pre_docs.append(clean_sent)
    
print("Completed removing stop words and cleaning the sentences...")
    

Loading stopwords for removal...
Completed removing stop words and cleaning the sentences...


In [3]:
ind = 0

docs = []
y_class = []

print("Initiated trimming long sentences to size of 50....")

for sent in pre_docs:
    if(len(sent.split())>50):
        temp_actual = sent.split()
        while (len(temp_actual)>50):
            sentence = ""
            temp_list = temp_actual[:50]
            for temp_words in temp_list:
                sentence += temp_words + " "
            temp_actual = temp_actual[50:]
            docs.append(sentence)
            y_class.append(classes[ind])
        if(len(temp_actual)>0):
            sentence = ""
            temp_list = temp_actual[:50]
            for temp_words in temp_list:
                sentence += temp_words + " "
            docs.append(sentence)
            y_class.append(classes[ind])
    else:
        docs.append(sent)
        y_class.append(classes[ind])
    ind += 1

print("Completed trimming...")

Initiated trimming long sentences to size of 50....
Completed trimming...


In [4]:
def category_y_class(age_as_int):
    if age_as_int >= 0 and age_as_int <= 13:
        return [1,0,0,0,0]
    elif age_as_int >= 14 and age_as_int <= 23:
        return [0,1,0,0,0]
    elif age_as_int >= 24 and age_as_int <= 45:
        return [0,0,1,0,0]
    elif age_as_int >= 46 and age_as_int <= 65:
        return [0,0,0,1,0]
    else:
        return [0,0,0,0,1]
    
class_labels = []

print("Initiated creating label class array...")

for entry in y_class:
    class_labels.append(category_y_class(entry))
    
print("Class label class array creation completed...")

#print(len(docs)," - ",len(class_labels))

Initiated creating label class array...
Class label class array creation completed...


In [5]:
print("Importing tensorflow libraries...")
import tensorflow as tffrom 
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Flatten, Conv1D
from tensorflow.keras.layers import Embedding

labels = np.array(class_labels)
vocab_size = 2000

print("Encoding given words to values...")

encoded_docs = [one_hot(d, vocab_size) for d in docs]
#print(encoded_docs)
max_length = 50
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
print("Defining the CNN model. Summary is provided below")
model = Sequential()
model.add(Embedding(vocab_size, 5, input_length=max_length))
model.add(Dropout(0.2))
model.add(Conv1D(filters=5,kernel_size=3,padding='valid', activation='relu', strides=1))
model.add(Flatten())
model.add(Dense(5, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())
# fit the model
print("Fitting the model with data...")
model.fit(padded_docs, labels, epochs=100, verbose=1)
# evaluate the model
print("Calculating accuracy...")
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))


Importing tensorflow libraries...
Encoding given words to values...
Defining the CNN model. Summary is provided below
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 5)             10000     
_________________________________________________________________
dropout (Dropout)            (None, 50, 5)             0         
_________________________________________________________________
conv1d (Conv1D)              (None, 48, 5)             80        
_________________________________________________________________
flatten (Flatten)            (None, 240)               0         
_________________________________________________________________
dense (Dense)                (None, 5)                 1205      
Total params: 11,285
Trainable params: 11,285
Non-trainable params: 0
_________________________________________________________________


Epoch 66/100
4276/4276 [==============================] - 1s 148us/sample - loss: 0.0203 - accuracy: 0.9934
Epoch 67/100
4276/4276 [==============================] - 1s 136us/sample - loss: 0.0191 - accuracy: 0.9939
Epoch 68/100
4276/4276 [==============================] - 1s 143us/sample - loss: 0.0179 - accuracy: 0.9944
Epoch 69/100
4276/4276 [==============================] - 1s 132us/sample - loss: 0.0200 - accuracy: 0.9942
Epoch 70/100
4276/4276 [==============================] - 1s 131us/sample - loss: 0.0187 - accuracy: 0.9941
Epoch 71/100
4276/4276 [==============================] - 1s 132us/sample - loss: 0.0190 - accuracy: 0.9943
Epoch 72/100
4276/4276 [==============================] - 1s 157us/sample - loss: 0.0183 - accuracy: 0.9947
Epoch 73/100
4276/4276 [==============================] - 1s 170us/sample - loss: 0.0198 - accuracy: 0.9935
Epoch 74/100
4276/4276 [==============================] - 1s 139us/sample - loss: 0.0192 - accuracy: 0.9941
Epoch 75/100
4276/4276 [====

In [6]:
model.save("AgeCategory_cnn_model.model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: AgeCategory_cnn_model.model\assets


In [27]:
def max_ind_fn(output_list):
    ind = 0
    max_ind = 0
    key = output_list[0]
    for item in output_list:
        if(key<item):
            key = item
            max_ind = ind
        ind+=1
    return max_ind

inp_str = np.array([padded_docs[185]])

result = model.predict(inp_str)

print("Given string belongs to Age-Category: ",categories[max_ind_fn(result[0])])

Given string belongs to Age-Category:  teenager
